In [16]:
import pandas as pd
import numpy as np
import data_handler as dh
#import model as md
import time
import matplotlib.pyplot as plt
from matplotlib import rc
import pickle

In [17]:
import importlib
importlib.reload(dh)
importlib.reload(md)

<module 'model' from '/Users/Dima/GAS/model.py'>

In [18]:
import numba
numba.__version__

'0.46.0'

In [19]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
from platform import python_version
print('python', python_version())
print('numpy', np.__version__)
print('pandas', pd.__version__)
import graph_tool
print('graph_tool', graph_tool.__version__)

/Users/Dima/anaconda3/bin/python
3.7.5 (default, Oct 25 2019, 10:52:18) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
sys.version_info(major=3, minor=7, micro=5, releaselevel='final', serial=0)
python 3.7.5
numpy 1.17.4
pandas 0.25.2
graph_tool 2.29 (commit , )


In [23]:
import copy
import numpy as np
import transport_graph as tg

import oracles
import dual_func_calculator as dfc

#import universal_similar_triangles_method as ustm
#import universal_gradient_descent_method as ugd
#import subgradient_descent_method as sd
#import frank_wolfe_method as fwm
#import weighted_dual_averages_method as wda
#import accelerated_metaalgorithm_method as am

In [24]:
import numpy as np
from history import History

# Модель Бэкмана

In [25]:
beckmann_save = 'beckmann_results/'

net_name = 'Anaheim_net.tntp'
trips_name = 'Anaheim_trips.tntp'

mu = 0.25
rho = 0.15

handler = dh.DataHandler()
graph_data = handler.GetGraphData(net_name, columns = ['init_node', 'term_node', 'capacity', 'free_flow_time'])
graph_correspondences, total_od_flow = handler.GetGraphCorrespondences(trips_name)



table = graph_data['graph_table'].copy()
inits = np.unique(table['init_node'][table['init_node_thru'] == False])
terms = np.unique(table['term_node'][table['term_node_thru'] == False])
through_nodes = np.unique([table['init_node'][table['init_node_thru'] == True], 
                           table['term_node'][table['term_node_thru'] == True]])

nodes = np.concatenate((inits, through_nodes, terms))
nodes_inds = list(zip(nodes, np.arange(len(nodes))))
init_to_ind = dict(nodes_inds[ : len(inits) + len(through_nodes)])
term_to_ind = dict(nodes_inds[len(inits) : ])

table['init_node'] = table['init_node'].map(init_to_ind)
table['term_node'] = table['term_node'].map(term_to_ind)
correspondences = {}
for origin, dests in graph_correspondences.items():
    dests = copy.deepcopy(dests)
    correspondences[init_to_ind[origin]] = {'targets' : list(map(term_to_ind.get , dests['targets'])), 
                                                             'corrs' : dests['corrs']}
    
inds_to_nodes = dict(zip(range(len(nodes)), nodes))
graph_correspondences = correspondences
graph_table = table
graph = tg.TransportGraph(graph_table, len(inds_to_nodes), graph_data['links number'])    
    
    
phi_big_oracle = oracles.PhiBigOracle(graph, graph_correspondences)
h_oracle = oracles.HOracle(graph.freeflow_times, graph.capacities, 
                           rho = rho, mu = mu)
primal_dual_calculator = dfc.PrimalDualCalculator(phi_big_oracle, h_oracle,
                                graph.freeflow_times, graph.capacities,
                                rho = rho, mu = mu, base_flows = None)    

Начальные установки, объявление функций.

In [15]:
oracle = phi_big_oracle
prox = h_oracle.prox
primal_dual_oracle = primal_dual_calculator

x_start = graph.freeflow_times
L_init = graph.max_path_length**0.5 * total_od_flow
max_iter = 500
eps = 1e-5
eps_abs = None
stop_crit = 'max_iter'
verbose_step = 100
verbose = False
save_history = True

Итерационный процесс.

In [9]:
tic = time.time()




if stop_crit == 'max_iter':
    def crit():
        return it_counter == max_iter
else:
    raise ValueError("stop_crit should be 'max iter'")

    
flows_start = primal_dual_oracle.get_flows(x_start) 
primal, dual, duality_gap_init, state_msg = primal_dual_oracle(flows_start, x_start)


L_value = L_init if L_init is not None else np.linalg.norm(oracle.grad(x_start))
#L_value = 10


want_iter = max_iter
podgonka = want_iter / 100

H = 2*L_value*podgonka
K = max_iter
A = 0.0
A_prev = A
a = 0.0
x = np.copy(x_start)
y = np.copy(x)
x_int = np.copy(x)

end = 0


if save_history:
    history = History('iter', 'primal_func', 'dual_func', 'dual_gap')
    history.update(0, primal, dual, duality_gap_init)
if verbose:
    print(state_msg)
if eps_abs is None:
    eps_abs = eps * duality_gap_init

success = False    



for it_counter in range(1, K+1):
    
    A_prev = A
    x_prev = np.copy(x)
    y_prev = np.copy(y)
    
#    print(f'A_prev:\t{A_prev}')
#    print(f'x_prev:\n {x_prev[:10]}')
#    print(f'y_prev:\n {y_prev[:10]}')
    
    lam = 0.5/H
    a = (lam + (lam**2 + 4*lam*A_prev)**0.5)/2
    A = A_prev + a
    x_int = (A_prev/A)*y_prev + (a/A)*x_prev
#    x_int = np.maximum((A_prev/A)*y_prev + (a/A)*x_prev, 0)


#    print(f'lambda:\t{lam}')
#    print(f'a:\t{a}')
#    print(f'A:\t{A}')
#    print(f'x_int:\n {x_int[:10]}')

    
    y = prox(oracle.grad(x_int), x_int, H)
#    y = np.maximum(prox(oracle.grad(x_int), x_int, H), 0)
#    y = np.ones(len(x_start))
    flows = primal_dual_oracle.get_flows(y)
    x = x_prev - a*flows - a*h_oracle.grad(y)

    primal, dual, duality_gap, state_msg = primal_dual_oracle(flows, y)
    if save_history:
        history.update(it_counter, primal, dual, duality_gap)
    if verbose and (it_counter % verbose_step == 0):
        print('\nIterations number: {:d}'.format(it_counter))
        print(state_msg, flush = True)
    if crit():
        success = True
        break
        
        
    
    if (sum(y - x_start) == 0) and end == 0:
        print(f'iter:\t{it_counter}\ttimes is equal to t_start')
        end = 1
        
    if it_counter % 100 == 0:
        print(f'iter:\t{it_counter}')
        
    if end == 1 and sum(y - x_start) != 0:
        print(f'{it_counter}||y-x||:\t{(sum((y-x)**2 for y, x in zip(y, x_start)))**0.5}')
   


    
result = {'times': x, 'flows': flows,
          'iter_num': it_counter,
          'res_msg': 'success' if success else 'iterations number exceeded'}
if save_history:
    result['history'] = history.dict
if verbose:
    print('\nResult: ' + result['res_msg'])
    print('Total iters: ' + str(it_counter))
    print(state_msg)
    print('Oracle elapsed time: {:.0f} sec'.format(oracle.time))
    
    
    
    
toc = time.time()

iter:	13	times is equal to t_start
iter:	100
111||y-x||:	5.289102489314246e-13
112||y-x||:	3.419486915845482e-14
113||y-x||:	2.3314683517128287e-15
114||y-x||:	2.220446049250313e-16
119||y-x||:	1.1102230246251565e-16
121||y-x||:	6.73594513500575e-12
122||y-x||:	4.4864112425102576e-13
123||y-x||:	3.26405569239796e-14
124||y-x||:	2.7755575615628914e-15
125||y-x||:	3.3306690738754696e-16
127||y-x||:	1.1102230246251565e-16
129||y-x||:	2.794704467845577e-10
130||y-x||:	1.8639534360431753e-11
131||y-x||:	1.355138223857466e-12
132||y-x||:	1.155742168634788e-13
133||y-x||:	1.2878587085651816e-14
134||y-x||:	2.1094237467877974e-15
135||y-x||:	5.551115123125783e-16
136||y-x||:	2.220446049250313e-16
137||y-x||:	1.1102230246251565e-16
138||y-x||:	1.1102230246251565e-16
139||y-x||:	1.1102230246251565e-16
140||y-x||:	1.1102230246251565e-16
142||y-x||:	2.220446049250313e-16
143||y-x||:	3.4122353831378405e-09
144||y-x||:	2.3059787412904598e-10
145||y-x||:	1.6880052911005805e-11
146||y-x||:	1.465938481

377||y-x||:	2.220446049250313e-16
378||y-x||:	1.1102230246251565e-16
380||y-x||:	1.1102230246251565e-16
381||y-x||:	1.1102230246251565e-16
383||y-x||:	2.220446049250313e-16
384||y-x||:	1.1102230246251565e-16
386||y-x||:	1.1102230246251565e-16
387||y-x||:	1.1102230246251565e-16
389||y-x||:	2.220446049250313e-16
390||y-x||:	1.1102230246251565e-16
392||y-x||:	1.1102230246251565e-16
393||y-x||:	1.1102230246251565e-16
395||y-x||:	1.1102230246251565e-16
396||y-x||:	1.1102230246251565e-16
398||y-x||:	2.220446049250313e-16
399||y-x||:	1.1102230246251565e-16
iter:	400
401||y-x||:	1.1102230246251565e-16
402||y-x||:	1.1102230246251565e-16
404||y-x||:	1.1102230246251565e-16
405||y-x||:	1.1102230246251565e-16
407||y-x||:	1.1102230246251565e-16
408||y-x||:	1.1102230246251565e-16
410||y-x||:	1.1102230246251565e-16
412||y-x||:	2.220446049250313e-16
413||y-x||:	1.1102230246251565e-16
415||y-x||:	1.1102230246251565e-16
416||y-x||:	1.1102230246251565e-16
418||y-x||:	1.1102230246251565e-16
420||y-x||:	2.2

In [10]:
print('Elapsed time: {:.0f} sec'.format(toc - tic))
print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, end = '\n\n')

Elapsed time: 66 sec
Time ratio = 1.0
Flow excess = 1.652111111111111

